<a href="https://colab.research.google.com/github/saskiabr/Thesis-SB/blob/main/GPT_anesthesiology_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to ChatGPT

In [ ]:
import openai
from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key = userdata.get('OPENAI_API_KEY')
,
)

## Assistant files

In [ ]:
questionnaire = client.files.create(
    file=open("questionnaire.xlsx", "rb"),
    purpose='assistants',
)

In [ ]:
json = client.files.create(
    file=open("rep_struct_shots.json", "rb"),
    purpose='assistants',
)

In [ ]:
ontology = client.files.create(
    file=open("onto.xml", "rb"),
    purpose='assistants',
)

## Assistant

In [ ]:
assistant = client.beta.assistants.create(
  name="Anesthesiologist helper",
  instructions=f"Imagine you are an anesthesiologist at the University Medical Center of Utrecht. Consider the context: when patients are scheduled for surgery, they will first fill in a general health questionnaire, and then have a preoperative screening session with another anesthesiologist. After such a preoperative screening session, a transcript will be created.\n You are provided three files (files are in Dutch): \n1. An excel file containing the results of the general health questionnaire filled in by all patients ({questionnaire.id}). \n2. A JSON file containing the required structure of medical reports ({json.id}). \n3. An ontology file containing relevant medical information (ontology is in RDF/XML format)({ontology.id}).\nWhen the user provides you with a transcript, please create a Dutch medical report for it using the structure specified in the JSON file. Important: use the structure exactly: no additional fields may be added; if no information can be found the field must be left empty. The medical ontology can be used to inform the answers to questions in the report. Only consider the information provided in the transcript or provided by the patient in their answers to the general health questionnaire when creating the report. Consider that the report is used for communication between doctors who use abbreviations, keywords and short sentences.",
  model="gpt-4o",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [json.id, questionnaire.id, ontology.id]
    }
  }
)

## Send message

In [ ]:
transcript = client.files.create(
    file=open("Patient_224.txt", "rb"),
    purpose='assistants',
)

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": f"Create a medical report based on this transcript text file of the conversation with patient 224 ({transcript.id}). Use the results of the questionnaire of patient 224 (found in column “Opname”). Do not log intermediate steps. Only return the final structured medical report.",
      "attachments": [
        {
          "file_id": transcript.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

## Run

In [ ]:
messages = list()

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)

if run.status == 'completed':
  new_messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  messages=new_messages
else:
  print(run.status)

In [ ]:
for message in messages:
    if message.role == "assistant":
        print("Answer: ",message.content[0].text.value)  # Print only the assistant's response

Answer:  ```json
{
    "Opname": "Patient 224",
    "Type consult": "fysiek",
    "Hoofdverrichting": "niet gespecificeerd",
    "Allergieën": "",
    "Medicatie": "",
    "Roken": "nee",
    "Alcohol": "ja",
    "Drugs": "nee",
    "Cardiale voorgeschiedenis": "cardiaal blanco",
    "Pulmonaal": "geen bijzonderheden",
    "Nierfunctie": "normaal",
    "Leverfunctie": "normaal",
    "Diabetes": "nee",
    "Neurologisch": "geen bijzonderheden",
    "Psychiatrisch": "geen bijzonderheden",
    "Infectieus": "geen bijzonderheden",
    "Hematologisch": "geen bijzonderheden",
    "Endocrien": "geen bijzonderheden",
    "Reumatologisch": "geen bijzonderheden",
    "Gastro-intestinaal": "geen bijzonderheden",
    "Overig": "geen bijzonderheden",
    "Voorgeschiedenis": "geen bijzonderheden",
    "Lichamelijk onderzoek": "geen bijzonderheden",
    "Auscultatie hart": "normaal",
    "Auscultatie longen": "normaal",
    "Carotiden": "geen souffle",
    "Craniofaciale abnormaliteiten": "nee",
    